In [90]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import random
import os
from sklearn.metrics.pairwise import cosine_similarity


In [20]:
import datetime

now = datetime.datetime.now()


Today_time = now.strftime("%H:%M")

Today_date = now.strftime("%Y-%m-%d")

In [21]:
from bs4 import BeautifulSoup
import nltk
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [22]:
nltk.download('stopwords')
nltk.download('punkt')
stopset = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\drago\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\drago\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
import nltk
import re

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\drago\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
def tokenize_and_stem(text):
   
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []

    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):

    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
   
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [25]:
prefix_dir =r"C:\Users\drago\Documents\GitHub\Summer-Project\Entertainment\More from Entertainment\Combined CSV\ "
suffix_dir = 'combined_entertainment-'+Today_date+'.csv'
today_csv=prefix_dir[:-1] +suffix_dir

In [27]:
ent = pd.read_csv(today_csv)

In [47]:
len(ent[ent['Category']=='gossip'])

12

In [29]:
ent['Category'].unique()

array(['bollywood', 'hollywood', 'movie-review', 'regional', 'television',
       'music', 'telugu', 'tamil', 'malayalam', 'web-series', 'celebs',
       'celebrities', 'regional-cinema', 'gossip', 'star-kids'],
      dtype=object)

In [111]:
df_dict = {}
result_dict = {}
for x in ent['Category'].unique():
    
    temp = ent[ent['Category']==x]
    df_dict[x] = temp
    Heading = temp['Heading']
    vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,4))
    
    tfidf_matrix = vectorizer.fit_transform(Heading)
    dist = cosine_similarity(tfidf_matrix)
    
    #n = get_best_n(tfidf_matrix,len(df_dict[x]),dist)
    if(len(temp)<=15):
        n=len(temp)-1
    else:
        n=15
    km = KMeans(n_clusters=n)
    km.fit(tfidf_matrix)
    

<6x6 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [30]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd

In [95]:
def get_best_n(tfidf_matrix,x,dist):
    n_clusters = list (range (5,x-1))
    min=999
    for n in n_clusters:
        km_ss = KMeans(n_clusters=n)
        clusters_ss = km_ss.fit_predict(tfidf_matrix)
        score = silhouette_score(dist,clusters_ss)
        if min>score:
            min=score
            n_score=n
    return n_score

In [138]:
def cluster_similarity(cluster):
    
    try:
        x=[]

        cluster_heading=cluster['Heading']
        #print(cluster['Heading'])
        cluster_matrix = vectorizer.fit_transform(cluster_heading)
        cluster_dist = cosine_similarity(cluster_matrix)
        cluster_elements_count = pd.DataFrame.count(cluster)
        #print(cluster_elements_count[0])


        for i in cluster_dist:
            #z=(i.sum()-1)/cluster_elements_count
            if((cluster_elements_count[0]-1)==0):
                y=1
            else:
                y=float("{0:.2f}".format(((i.sum())/(cluster_elements_count[0]))))
            x.append(y)
            cluster_similarity_value.append(y)

        
        
    except:
        for i in range(len(cluster['Heading'])):
            x.append(1)
            cluster_similarity_value.append(1)
    
    return x       

In [135]:
temp_more =[]
temp_less  =[]
def fix_you(cluster):
    cluster_mean = cluster['cluster_similarity_value'].mean()
    for i in range(len(cluster)):
        if (cluster.iloc[i]['cluster_similarity_value']<cluster_mean):
            temp_less.append(cluster.iloc[i])
        else:
            temp_more.append(cluster.iloc[i])

In [137]:
df_dict = {}
for x in ent['Category'].unique():
    
    temp = ent[ent['Category']==x]
    Heading = temp['Heading']
    vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,4))
    tfidf_matrix = vectorizer.fit_transform(Heading)
    
    dist = cosine_similarity(tfidf_matrix)

    #n = get_best_n(tfidf_matrix,len(temp),dist)
    if(len(temp)<=15):
        n=len(temp)-1
    else:
        n=15
    km = KMeans(n_clusters=n)
    km.fit(tfidf_matrix)
    
    clusters = km.labels_.tolist()
    temp['Cluster'] = clusters
 
    df_sorted=temp.sort_values(by='Cluster').reset_index()
    df_sorted.drop(['index'],axis=1,inplace=True)
    
    grp = df_sorted.sort_values('Cluster').groupby(['Cluster'],as_index=False)

    cluster_similarity_value =[];temp_more =[];temp_less  =[]
    for i in range(n):
        cs  = cluster_similarity(grp.get_group(i))
        
        cluster_similarity_value.append(cs)
    df_sorted['cluster_similarity_value']=cluster_similarity_value;  col=df_sorted.columns
    cluster_similarity_value = []
    for i in range(n):
        fix_you(grp.get_group(i))
    
    #df_more_similar=pd.DataFrame(temp_more,columns=col)
    #df_less_similar=pd.DataFrame(temp_less,columns=col)
    
    
    #Result = df_more_similar.sort_values('Cluster').groupby(['Cluster'],as_index=False).apply(lambda x: x.sample(1)) 
    #Result = Result.reset_index().drop(['level_0','level_1'],axis=1)
    #Result = Result.append(df_less_similar)
    #Result = Result.sort_values(by='Cluster')
    #Result = Result.reset_index().drop(['index'],axis=1)
    
    #df_dict[x] = Result
    

[1]
[0.1, 0.1, 0.1, 0.12, 0.1, 0.1, 0.11, 0.11, 0.12, 0.11]
[0.23, 0.21, 0.21, 0.24, 0.27]
[0.16, 0.21, 0.19, 0.22, 0.18, 0.15, 0.17]
[0.33, 0.39, 0.39]
[0.21, 0.22, 0.24, 0.24, 0.24]
[0.37, 0.37, 0.33]
[0.47, 0.33, 0.47]
[0.25, 0.28, 0.31, 0.32]
[0.34, 0.35, 0.34]
[0.17, 0.18, 0.18, 0.18, 0.22, 0.21]
[0.5, 0.5]
[0.31, 0.27, 0.26, 0.3]
[0.1, 0.1, 0.11, 0.14, 0.1, 0.13, 0.11, 0.14, 0.12, 0.13]
[0.22, 0.22, 0.2, 0.21, 0.21]


ValueError: Length of values does not match length of index

In [13]:
pd.DataFrame.rename

,Newspaper,Heading,Category,Date,Time,URL
0,Hindiustan Times,Suhana Khan stuns in latest mirror selfie in a...,bollywood,2019-06-05,15:18:00,https://www.hindustantimes.com/bollywood/suhan...
1,Hindiustan Times,"Bharat movie review: Salman Khan, Katrina Kaif...",bollywood,2019-06-05,15:03:00,https://www.hindustantimes.com/bollywood/bhara...
2,Hindiustan Times,Panga: Kangana Ranaut feasts on South Indian f...,bollywood,2019-06-05,13:50:00,https://www.hindustantimes.com/bollywood/panga...
3,Hindiustan Times,Super 30: Trailer of the Hrithik Roshan film g...,bollywood,2019-06-05,13:49:00,https://www.hindustantimes.com/bollywood/super...
4,Hindiustan Times,Bharat advance booking: Salman Khan’s film ran...,bollywood,2019-06-05,13:47:00,https://www.hindustantimes.com/bollywood/bhara...
5,Hindiustan Times,"Actor Dinyar Contractor, known for his roles i...",bollywood,2019-06-05,12:29:00,https://www.hindustantimes.com/bollywood/actor...
6,Hindiustan Times,Mira Rajput’s son Zain pinches sister Misha as...,bollywood,2019-06-05,11:48:00,https://www.hindustantimes.com/bollywood/mira-...
7,Hindiustan Times,Katrina Kaif reveals she had one condition bef...,bollywood,2019-06-05,11:39:00,https://www.hindustantimes.com/bollywood/katri...
8,Hindiustan Times,Sara Ali Khan twins with mom Amrita in green a...,bollywood,2019-06-05,10:53:00,https://www.hindustantimes.com/bollywood/sara-...
9,Hindiustan Times,"Tiger Shroff, Abhishek Bachchan and other Boll...",bollywood,2019-06-05,10:55:00,https://www.hindustantimes.com/bollywood/tiger...
